In [1]:
# << 파이썬으로 이메일 전송하기 >>

# 조건 : 네이버 - 로그인 2단계 인증을 사용한다면 해제 되어있어야 함

In [7]:
import smtplib

In [8]:
from email.mime.text import MIMEText

In [9]:
# 시스템 라이브러리 활용 패키지
import os

# 이메일 메시지에 형식을 활용하기 위한 패키지
from email.mime.multipart import MIMEMultipart

# 이메일 메시지 프로토콜 처리기(엔코더)
from email import encoders

# 텍스트 형식 메일
from email.mime.text import MIMEText

# 이미지 형식 메일
from email.mime.image import MIMEImage

# 오디오 형식 메일
from email.mime.audio import MIMEAudio

# 이 모든 정보들을 관장하는 객체
from email.mime.base import MIMEBase

In [10]:
def send_email(smtp_info, msg):
    with smtplib.SMTP(smtp_info["smtp_server"], smtp_info["smtp_port"]) as server:
        # TLS 보안 설정 및 시작 (보안 정책상 필수)
        server.starttls()
        
        # 로그인
        server.login(smtp_info["smtp_user_id"], smtp_info["smtp_user_pw"])
        
        # 이메일 전송
        # 메시지 전송시 반드시 as_string()으로 변환하여 전송해야함
        res = server.sendmail(msg['from'], msg['to'], msg.as_string())
        
        if not res:
            print('이메일 전송 성공!!!')
        else:
            print(res)

In [17]:
# << 텍스트, 이미지 보내기 >>

def make_multimsg(msg_dict):
    multi = MIMEMultipart(_subtype = 'mixed')
    
    for key, value in msg_dict.items():
        # text 형식인 경우
        if key == 'text':
            with open(value['filename'], encoding='utf-8') as fp:
                msg = MIMEText(fp.read(), _subtype = value['subtype'])
        # image 형식인 경우
        elif key == 'image':
            with open(value['filename'], 'rb') as fp:
                msg = MIMEImage(fp.read(), _subtype = value['subtype'])
        # audio, video는 별도 설정 없이 진행하도록 함
        elif key == 'audio':
            print("음원은 지원하지 않습니다!")
        elif key == 'video':
            print("영상은 지원하지 않습니다!")
        # 그 외의 형식
        else:
            with open(value['filename'], 'rb') as fp:
                msg = MIMEBase(value['maintype'], _subtype = value['subtype'])
                msg.set_payload(fp.read())
                encoders.encode_base64(msg)
                
        # 파일 이름은 첨부파일 제목
        msg.add_header('Content-Disposition', 'attachment', 
                       filename = os.path.basename(value['filename']))

        # 파일 첨부!
        multi.attach(msg)
        
    return multi

In [14]:
# 텍스트만 전송하기
smtp_info = dict({
    "smtp_server": "smtp.naver.com",
    "smtp_user_id": "03y20y@naver.com",
    "smtp_user_pw": "네이버 비번",
    "smtp_port": 587
})

title = "쪼코야 잘 지내니"
content = "잘 지내렴"
sender = smtp_info['smtp_user_id']
receiver = smtp_info['smtp_user_id']

msg = MIMEText(_text = content, _charset = "utf-8")

msg['Subject'] = title
msg['From'] = sender
msg['To'] = receiver

send_email(smtp_info, msg)

이메일 전송 성공!!!


# SMTP(Simple Mail Transfer Protocol)

POP3(Post Office Protocol 3), IMAP(Internet Message Access Protocol)의 약자
프로토콜이란? 상호 조약 같은 것으로 보면 됩니다.
국제 표준 언어가 영어이듯이 서로 다른 언어를 사용하는 녀석들끼리 표준을 만들어 소통을 하기 위함임

1. 송신자 계정으로 로그인 한다. (ID, PW 활용)
2. 이메일 작성(제목, 내용, 이미지 등등 첨부가 가능함)
3. send를 통해 이메일 서버에 전송을 요청함
4. 이메일 서버가 SMTP, POP3, IMAP 프로토콜을 활용하여 수신자에게 메일을 전송하게 됨

In [20]:
# 텍스트, 이미지(첨부파일) 전송하기
smtp_info = dict({
    "smtp_server": "smtp.naver.com",
    "smtp_user_id": "03y20y@naver.com",
    "smtp_user_pw": "네이버 비번",
    "smtp_port": 587
})

msg_dict = {
    # 텍스트 형식의 첨부 파일
    'text': {
        'maintype': 'text',
        'subtype': 'plain',
        'filename': 'res/email/test.txt'
    },
    # 이미지 형식의 첨부 파일
    'image': {
        'maintype': 'image',
        'subtype': 'jpg',
        'filename': 'res/email/bonobono.jpg'
    },
    # 세금계산서 등등의 pdf 형식의 첨부 파일들
    'application': {
        'maintype': 'application',
        'subtype': 'octect-stream',
        'filename': 'res/email/test.pdf'
    }
}
# 메일 작성
title = "첨부파일을 보냈습니다."
content = "확인해보세요"
sender = smtp_info['smtp_user_id']
receiver = smtp_info['smtp_user_id']

msg = MIMEText(_text = content, _charset = 'utf-8')

# 파일 첨부하기!
multi = make_multimsg(msg_dict)
multi['Subject'] = title
multi['From'] = sender
multi['To'] = receiver
multi.attach(msg)

# 첨부한 내용을 추가하여 이메일 전송!
send_email(smtp_info, multi)

이메일 전송 성공!!!
